In [ ]:
import os
import time

from jax import numpy as np
import numpy.random as npr

import matplotlib
import matplotlib.animation 
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 1024

import skimage
import skimage.io as sio
import skimage.transform
import fracatal

from fracatal.functional_jax import ft_convolve, \
        make_gaussian, \
        make_mixed_gaussian, \
        make_kernel_field, \
        make_update_function, \
        make_transition_function, \
        make_update_step, \
        make_make_kernel_function, \
        sigmoid_1, \
        get_smooth_steps_fn, \
        make_make_smoothlife_kernel_function, \
        make_smooth_interval, \
        compute_entropy, \
        compute_frequency_ratio, \
        compute_frequency_entropy, \
        make_smoothlife_update_function, \
        make_smoothlife_update_step
        
from fracatal.scripts import v_stability_sweep, stability_sweep     

import IPython

In [ ]:
"""
animation functions
"""

def get_fig(grid):
    
    global subplot_0
    
    fig, ax = plt.subplots(1,1)
    
    subplot_0 = ax.imshow(grid.squeeze(), cmap="magma")
    
    return fig, ax

def update_frame(ii):
    
    global grid
    
    subplot_0.set_array(grid.squeeze())
    
    grid = update_step(grid)

In [ ]:
! ls ../patterns

In [ ]:
#pattern_name = "scutium_gravidus_single"
#pattern_name = "orbium_orbium"
#pattern_name = "hydrogeminium_natans_pickle"
pattern_name = "hydrogeminium_natans_wobbler"
#pattern_name = "hydrogeminium_natans_crispus_31"

In [ ]:
#### kernels

if "orbium" in pattern_name:
    #o. unicaudatus
    # the neighborhood kernel
    amplitudes = [1.0]
    means = [0.5]
    standard_deviations = [0.15]
    k0 = 13
elif "hydrogeminium_natans" in pattern_name:
    # H. natans
    
    # the neighborhood kernel
    amplitudes = [0.5, 1.0, 0.6667]
    means = [0.0938, 0.2814, 0.4690]
    standard_deviations = [0.0330, 0.0330, 0.0330]
    k0 = 31
elif "scutium_gravidus" in pattern_name:
    
    amplitudes = [1.0]
    means = [0.5]
    standard_deviations = [0.15]
    k0 = 13

#### Growth functions

if "orbium" in pattern_name:
    # O. unicaudatus
    mean_g = 0.15
    standard_deviation_g = 0.017
elif "hydrogeminium_natans" in pattern_name:
    # H. natans
    mean_g = 0.26
    standard_deviation_g = 0.036
elif "scutium_gravidus" in pattern_name:
        
    mean_g = 0.283
    standard_deviation_g = 0.0369

In [ ]:
kernel_radius = k0
grid_dim = 128
num_frames = 32
dts = 0.5

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations,dim=122)
kernel = make_kernel(kernel_radius)

my_update = make_update_function(mean_g, standard_deviation_g)
clipping_fn = lambda x: np.clip(x, 0, 1.0)
update_step = make_update_step(my_update, kernel, dts, clipping_fn)

pattern_filepath = os.path.join("..", "patterns", f"{pattern_name}.npy")

pattern = np.load(pattern_filepath)

grid = np.zeros((1, 1, grid_dim, grid_dim))
grid = grid.at[:,:,45:45+pattern.shape[-2], 45:45+pattern.shape[-1]].set(pattern)

mass_0 = grid.sum().item()
relative_mass = [mass_0/mass_0]

for my_step in range(1024):
    grid = update_step(grid)
    mass_n = grid.sum().item()

    relative_mass.append(mass_n / mass_0)

relative_mass = np.array(relative_mass)
print(pattern_name)
print(f"mean +/- std. dev: {np.mean(relative_mass):.3f}+/-{np.std(relative_mass):.3f}")
print(f"min, max growth: {np.min(relative_mass):.3f}, {np.max(relative_mass):.3f} ")


fig, ax = get_fig(grid[0])
plt.close()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_frame, frames=num_frames, interval=10).to_jshtml())

In [ ]:
num_frames = 128
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_frame, frames=num_frames, interval=10).to_jshtml())

In [ ]:
temp = grid[:,:,43:68, 38:62]

plt.imshow(temp.squeeze())

np.save("../patterns/hydrogeminium_natans_crispus_31.npy", temp)

save_image = np.array(255*plt.get_cmap("magma")(temp.squeeze())[:,:,:3], dtype=np.uint8)

plt.imshow(save_image)
sio.imsave("../patterns/hydrogeminium_natans_crispus_31.png", save_image)

# Results 

## S. gravidus
scutium_gravidus_single
mean +/- std. dev: 1.004+/-0.007
min, max growth: 0.987, 1.025 

## O. unicaudatus
orbium_orbium
mean +/- std. dev: 1.061+/-0.004
min, max growth: 1.000, 1.079 

## h. natans pickle (dt 0.4)
hydrogeminium_natans_pickle
mean +/- std. dev: 0.997+/-0.004
min, max growth: 0.981, 1.011 

## h. natans wobbler (dt 0.4)
hydrogeminium_natans_wobbler
mean +/- std. dev: 1.096+/-0.019
min, max growth: 1.000, 1.186  

## h. natans crispus (dt 0.5)
hydrogeminium_natans_crispus_31
mean +/- std. dev: 1.001+/-0.004
min, max growth: 0.987, 1.015 

Note that the conversion of H. natans wobbler to H. natans crispus:
hydrogeminium_natans_wobbler
mean +/- std. dev: 0.863+/-0.026
min, max growth: 0.813, 1.306 